In [55]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [4]:
os.chdir('..')
os.getcwd()

'C:\\Users\\aniru\\Desktop\\Amrita\\Sem 4\\Mathematics for Intelligent Systems\\Project\\SpatioTemporalKoopman'

# Data Processing

In [5]:
pathData = os.path.join(os.getcwd(), 'data', 'BTC-USD.csv')

In [6]:
data = pd.read_csv(pathData)

In [8]:
data.drop(columns=['Adj Close', 'High', 'Low', 'Open'], inplace=True)

In [12]:
scaler = MinMaxScaler()
volD = data['Volume'].values.reshape(-1,1)
aa = scaler.fit_transform(volD)
data['Vol'] = pd.DataFrame(aa)

In [14]:
data.tail()

,Date,Close,Volume,Vol
2106,2021-05-13,49716.191406,9.672115e+10,0.275558
2107,2021-05-14,49880.535156,5.573750e+10,0.158780
2108,2021-05-15,46760.187500,5.916105e+10,0.168535
2109,2021-05-16,46456.058594,6.404787e+10,0.182460
2110,2021-05-17,45367.214844,8.320761e+10,0.237053


In [18]:
data['Date'][0]

'2015-08-07'

In [24]:
daa = data['Date'].values
mon = []
dat = []
for i in daa:
    mon.append(i[5:7])
    dat.append(i[8:])
data['Month'] = pd.DataFrame(mon)
data['Day'] = pd.DataFrame(dat)

In [25]:
data.head()

,Date,Close,Volume,Vol,Month,Day
0,2015-08-07,279.584991,42484800.0,0.000085,08,07
1,2015-08-08,260.997009,58533000.0,0.000131,08,08
2,2015-08-09,265.083008,23789600.0,0.000032,08,09
3,2015-08-10,264.470001,20979400.0,0.000024,08,10
4,2015-08-11,270.385986,25433900.0,0.000036,08,11


# Tensor Creation

In [69]:
def snap(dat):
    mon = dat['Month'].values
    dayy = dat['Day'].values
    vol = dat['Vol'].values
    close = dat['Close'].values
    snps = [[]]
    ctr = 0
    for i in range(1,(len(mon)-1)):
        if(mon[i-1] != mon[i]):
            snps.append(np.zeros(shape=(10**4,31)))
    for i in range(len(mon)):
        if(i != 0):
            if(mon[i-1] != mon[i]):
                ctr += 1
            
        snps[ctr][int(dayy[i])-1][int(vol[i]*10**6)] = close[i]
        print(snps)
    return snps

In [70]:
aa = snap(data)

IndexError: list index out of range